In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras as k
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import skimage.morphology as morp
from skimage.filters import rank

%matplotlib inline

assert pd.__version__ == '0.23.4'
assert np.__version__ == '1.15.4'
assert k.__version__ == '2.1.3'

In [ ]:
# Load all images. 

import os
import skimage.data
def load_data(data_dir):
    images = []
    
    file_names = sorted([os.path.join(data_dir, f)
                  for f in os.listdir(data_dir) if f.endswith(".jpg")])
    
    for f in file_names:
        images.append(skimage.data.imread(f, as_gray=True))
    
    return images

images = load_data('../data/train/cropped')
print(len(images))

In [ ]:
#explore shapes
for image in images[:3]:
    print('image.shape[before]', image.shape)

In [ ]:
#transform all images to 250x250

image_size = 64
import skimage.transform
images250 = [skimage.transform.resize(image, (image_size, image_size)) for image in images]

In [ ]:
#confirm transformation
for image in images250[:3]:
    print('image.shape[after]', image.shape)
    

In [ ]:
from PIL import Image

for image in images250[:3]:
    image = (image * 255).astype(np.uint8)
    im = Image.fromarray(image)
    plt.figure()
    plt.imshow(im)
    plt.show()



In [ ]:
#Load Labels 
data = pd.read_csv("../data/train.csv")
data = data.sort_values("Img_Name").reset_index(drop=True)
labels = data["Label"]

labels.head()

In [ ]:
#categorize labels..

import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# y_train_labels = np.array(labels)
# X_train_data = np.array(images250)
y = np.array(labels)
X = np.array(images250)
X = X[:, :, :, np.newaxis] # (250,250) --> (467,250,250,1)

num_categories = 6

#encode labels
label_encoder = LabelEncoder()
y_labels_encoded = label_encoder.fit_transform(y)
y_labels_categorized = to_categorical(y_labels_encoded, num_categories)

print('len(X)', len(X))
print('len(y)', len(y))
print('len(y_labels_categorized)', len(y_labels_categorized))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_labels_categorized, test_size=0.2, random_state=42)

In [ ]:
# OLD MODEL

# input_shape = (250, 250, 1)

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, data_format='channels_last'))
# # model.add(Conv2D(32, (3, 3), activation='relu'))
# # model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))

# # model.add(Conv2D(64, (3, 3), activation='relu'))
# # model.add(MaxPool2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))
# model.add(Flatten())

# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_categories, activation='softmax'))

# model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

# model.summary()

In [ ]:
# MODEL #1 -- Basic Non-convolutional

model = Sequential()
model.add(Flatten(input_shape=(image_size,image_size,1)))
model.add(Dense(units=image_size,activation="relu"))
model.add(Dense(units=image_size,activation="relu"))
model.add(Dense(units=6,activation="softmax"))
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32)

In [ ]:
accuracy = model.evaluate(x=X_test,y=y_test,batch_size=32)
accuracy

In [ ]:
# MODEL #2 -- Convolutional test 

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(image_size,image_size,1), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

# from keras.optimizers import SGD
# opt = SGD(lr=0.05)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop', #opt
              metrics=['accuracy']) 

# NOTE: If loss=categorical_crossentropy, 
# then 'accuracy' metrics == categorical_accruacy

model.summary()


In [ ]:
# data_gen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     zoom_range=0.2,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=False,
#     fill_mode="nearest",
#     channel_shift_range=0.2,
# )


# # compute quantities required for featurewise normalization
# # (std, mean, and principal components if ZCA whitening is applied)
# data_gen.fit(X_train)

# # fits the model on batches with real-time data augmentation:
# # model.fit_generator(train_gen.flow(X_train, y_train, batch_size=32),
# #                     steps_per_epoch=len(X_train) / 32, epochs=20)

# import os
# os.makedirs('../data/image_data_gen')

# i = 0
# for X_batch, y_batch in data_gen.flow(X_train, y_train, batch_size=5, save_to_dir='../data/image_data_gen', save_prefix='aug', save_format='jpg'):
#     i += 1
#     if i == 500:
#         break

# print("done.")

In [ ]:
train_gen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=False,
    fill_mode="nearest",
    channel_shift_range=0.2,
)

validation_gen = ImageDataGenerator(
    featurewise_center=False
#     featurewise_std_normalization=True
)


# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
train_gen.fit(X_train)
validation_gen.fit(X_train)

In [ ]:
epochs_per_round = 50
epoch_acc = []
for i in range(5):
    # fits the model on batches with real-time data augmentation:
    model.fit_generator(train_gen.flow(X_train, y_train, batch_size=32),
                        validation_data=validation_gen.flow(X_test, y_test, batch_size=32),
                        validation_steps=800 // 32,
                        steps_per_epoch=len(X_train) / 32, epochs=50,
                        verbose=0
                        )
    
    acc = accuracy = model.evaluate(x=X_test,y=y_test,batch_size=32)
    print("\nACCURACY AFTER ", epochs_per_round*(i+1), "EPOCHS:" acc, "\n\n\n")
    epoch_acc.append((acc, epochs_per_round*(i+1)))
    
print(epoch_acc)


In [ ]:
# model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
accuracy = model.evaluate(x=X_test,y=y_test,batch_size=32)
accuracy

In [ ]:
predictions = model.predict(X_test)
print('First prediction:', predictions[0])

predict_max = [max(class_probs) for class_probs in predictions]
print('Average predicted class probability:', np.mean(predict_max))

correct = [1 if np.argmax(predictions[i])==np.argmax(y_test[i]) else 0 for i in range(len(y_test))]
print("%-correct: ", np.mean(correct))

predictions[0:5]
